<center>
<img src='images/logo.jpg' width='300px'>
<h2> Chương 5: THEO VẾT VÀ NHẬN DẠNG ĐỐI TƯỢNG</h2>
</center>

---

A. Theo vết đối tượng:
Trong thị giác máy tính có 2 bài toán phổ biến:

* Phát hiện đối tượng - Object detection (OD) – Làm việc trên một khung hình (frame) 
* Theo vết đối tượng - Object tracking (OT) – Làm việc trên nhiều khung hình liên tiếp

Mục tiêu của Video Object tracking là xác định vị trí của một hoặc nhiều đối tượng (target) trong các frame liên tiếp của Video
* Single Object tracking (SOT) – Chỉ có một object được theo dõi ngay cả khi môi trường có nhiều  đối tượng.
* Muiltiple Object tracking (OT) – Nhiều đối tượng được theo dõi theo thời gian, thậm chí nó có thể theo dõi các đối tượng mới xuất hiện trong video.


## 1. Làm việc với Video
---


In [1]:
#Gọi thư viện:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [8]:
#Đọc video từ webcam và từ file 
#Kết nối webcam 
#cap = cv2.VideoCapture(0)

cap = cv2.VideoCapture("Video/slow_traffic_small.mp4")
if not cap.isOpened(): 
    print("Ko kết nối được với cam")
    exit()
#Hiển thị 
while True:
    #Đọc từng frame trong video 
    rec,frame = cap.read()

    cv2.imshow('Video WebCam',frame)
    #Thoát chế độ video 
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break


#Giải phóng bộ nhớ 
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In [5]:
#Tách lưu từng frame trong video: 

folder = 'Frames_output/'
count = 0 


cap = cv2.VideoCapture(0)
if not cap.isOpened(): 
    print("Ko kết nối được với cam")
    exit()


#Hiển thị 
while True:
    #Đọc từng frame trong video 
    rec,frame = cap.read()
    
    #Lưu các frame vào thư mục nhất định
    filename = folder + 'Frame_' + str(count) + '.jpg'
    cv2.imwrite(filename,frame)
    count += 1 
    

    
    cv2.imshow('Video WebCam',frame)
    #Thoát chế độ video 
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

    

#Giải phóng bộ nhớ 
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In [3]:
#Lưu video ra file
cap = cv2.VideoCapture(0)

if not cap.isOpened(): 
    print("Ko kết nối được với cam")
    exit()

#Thiết lập tham số ghi file Video:
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) + 0.5)
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) + 0.5)
size = (width, height)
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
out = cv2.VideoWriter('Result_object.mp4', fourcc, 20.0, size)


#Hiển thị 
while True:
    #Đọc từng frame trong video 
    rec,frame = cap.read()
    
    cv2.imshow('Video WebCam',frame)
    out.write(frame)
    #Thoát chế độ video 
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
    

out.release()
#Giải phóng bộ nhớ 
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

## 2. Một số phương pháp theo vết đối tượng
## 2.1 Optical Flow
---

Link tham khảo về optical flow:

https://nanonets.com/blog/optical-flow/

https://docs.opencv.org/master/d4/dee/tutorial_optical_flow.html

## a. Sparse Optical Flow
---
https://docs.opencv.org/3.4/d4/dee/tutorial_optical_flow.html

In [4]:
#Đọc video
cap = cv2.VideoCapture("Video/slow_traffic_small.mp4")

cv2.startWindowThread()
# params for ShiTomasi corner detection
st_params  = dict(maxCorners = 100, 
                  qualityLevel = 0.3,
                  minDistance = 7,
                  blockSize = 7)

# Parameters for lucas kanade optical flow
lk_params = dict(winSize  = (15,15),
                 maxLevel = 2,
                 criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,10, 0.03))

# Create some random colors
color = np.random.randint(0,255,(100,3))

# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)


p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **st_params)
# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)

while True:
    ret,frame = cap.read()
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
  
    # calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    
    # Select good points
    if p1 is not None:
        good_new = p1[st==1]
        good_old = p0[st==1]
    
    # draw the tracks
    for i,(new,old) in enumerate(zip(good_new, good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv2.line(mask, (int(a),int(b)),(int(c),int(d)), color[i].tolist(), 2)
        frame = cv2.circle(frame,(int(a),int(b)),5,color[i].tolist(),-1)
    img = cv2.add(frame,mask)
    cv2.imshow('Optical flow',img)
    
    
    if cv2.waitKey(300) & 0xFF == ord("q"):
        break
    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)
    
# Giải phóng bộ nhớ:
cap.release()

cv2.destroyAllWindows()    
cv2.waitKey(1)

-1

## b. Dense Optical Flow
---
Link tham khảo:
    
https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_video/py_lucas_kanade/py_lucas_kanade.html

In [5]:
#Import Libraries
import cv2
import numpy as np


# Đọc video:
cap = cv2.VideoCapture("Video/slow_traffic_small.mp4")

# Read the capture and get the first frame
ret,first_frame = cap.read()

# Convert frame to Grayscale
prev_gray = cv2.cvtColor(first_frame,
                        cv2.COLOR_BGR2GRAY)

# Create# Create an image with the same dimensions as the frame for later drawing purposes
mask = np.zeros_like(first_frame)

# Saturation to maximum
mask[...,1] = 255

# While loop
while True:

    # Read the capture and get the first frame
    ret,frame = cap.read()
    
    # Open new window and display the input frame
    cv2.imshow("input",frame)
    
    # Convert all frame to Grayscale (previously we did only the first frame)
    gray = cv2.cvtColor(frame,
                       cv2.COLOR_BGR2GRAY)
    
    # Calculate dense optical flow by Farneback
    flow = cv2.calcOpticalFlowFarneback(prev_gray,
                                       gray,
                                        None,
                                        0.5,
                                        3,15,3,5,1.2,0
                                       )
    
    # Compute Magnitude and Angle
    magn,angle = cv2.cartToPolar(flow[...,0],
                                 flow[...,1])
    
    # Set image hue depanding on the optical flow direction
    mask[...,0] = angle*180/np.pi/2    
        
    # Normalize the magnitude
    mask[...,2] = cv2.normalize(magn,
                                None,
                                0,
                                255,
                                cv2.NORM_MINMAX)
    
    # Convert HSV to RGB
    rgb = cv2.cvtColor(mask,cv2.COLOR_HSV2RGB)
    
    # Open new window and display the output
    cv2.imshow("Dense Optical Flow",rgb)
    
    # Update previous frame
    prev_gray = gray
    
    # Close the frame
    if cv2.waitKey(10) & 0XFF == ord('q'):
        break
    
# Release and Destroy
cap.release()

cv2.destroyAllWindows()
cv2.waitKey(1)

-1

## 2.2: MeanShift - CamShift
---
Link tham khảo:

https://docs.opencv.org/master/d7/d00/tutorial_meanshift.html

### a. MeanShift
---

In [6]:
#import libraries
import numpy as np
import cv2

In [7]:
#Read video
cap = cv2.VideoCapture("Video/slow_traffic_small.mp4")

# take first frame of the video
ret,frame = cap.read()

# setup initial location of window
x, y, w, h = 300, 200, 100, 50 # simply hardcoded the values
track_window = (x, y, w, h)

# set up the ROI for tracking
roi = frame[y:y+h, x:x+w]
hsv_roi =  cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv_roi, np.array((0., 60.,32.)), np.array((180.,255.,255.)))
roi_hist = cv2.calcHist([hsv_roi],[0],mask,[180],[0,180])
cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)

# Setup the termination criteria, either 10 iteration or move by atleast 1 pt
term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1 )
while True:
    ret, frame = cap.read()
    if ret == True:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)
        
        # apply meanshift to get the new location
        ret, track_window = cv2.meanShift(dst, track_window, term_crit)
        
        # Draw it on image
        x,y,w,h = track_window
        img2 = cv2.rectangle(frame, (x,y), (x+w,y+h), 255,2)
        cv2.imshow('Example MeanShift',img2)
        if cv2.waitKey(300) & 0xFF == ord("q"):
            break
    else:
        break

#Giải phóng bộ nhớ
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

## b. Camshift
---

In [8]:
#import libraries
import numpy as np
import cv2

In [9]:
#Read video
cap = cv2.VideoCapture("Video/slow_traffic_small.mp4")

# take first frame of the video
ret,frame = cap.read()

# setup initial location of window
x, y, w, h = 300, 200, 100, 50 # simply hardcoded the values
track_window = (x, y, w, h)

# set up the ROI for tracking
roi = frame[y:y+h, x:x+w]
hsv_roi =  cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv_roi, np.array((0., 60.,32.)), np.array((180.,255.,255.)))
roi_hist = cv2.calcHist([hsv_roi],[0],mask,[180],[0,180])
cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)

# Setup the termination criteria, either 10 iteration or move by atleast 1 pt
term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1 )
while True:
    ret, frame = cap.read()
    if ret == True:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)
        
        # apply camshift to get the new location
        ret, track_window = cv2.CamShift(dst, track_window, term_crit)
        # Draw it on image
        pts = cv2.boxPoints(ret)
        pts = np.int0(pts)
        img2 = cv2.polylines(frame,[pts],True, 255,2)
        cv2.imshow('Example CamShift',img2)
        if cv2.waitKey(300) & 0xFF == ord("q"):
            break
    else:
        break

#Giải phóng bộ nhớ:
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

C:\Users\nviet\AppData\Local\Temp\ipykernel_27964\3458440909.py:30: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  pts = np.int0(pts)


-1

# 3. Ví dụ : Theo vết một đối tượng trong Video (Single Object Tracking)
---

In [10]:
#Import Libraries
import cv2
import numpy as np
print('Version OpenCV:',cv2.__version__)

Version OpenCV: 4.11.0


In [11]:
# Capture the Video
cap = cv2.VideoCapture('Video/crime-object.mp4')

# Tracker sử dụng OpenCV:
tracker = cv2.TrackerCSRT_create() #CamShift
#tracker=cv2.TrackerMIL_create()     #MeanShift

# Đọc frame đầu tiên:
ret,frame = cap.read()

#Thiết lập tham số ghi file Video:
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) + 0.5)
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) + 0.5)
size = (width, height)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('Result_object.mp4', fourcc, 20.0, size)


# Select our ROI
roi = cv2.selectROI(frame,False)

# Initialize tracker
ret = tracker.init(frame,roi)

while True:
    # Read the capture
    ret,frame = cap.read()
    
    # update tracker
    success, roi = tracker.update(frame)
    
    # roi -> from tuple to int
    (x,y,w,h)=tuple(map(int,roi))
    
    # Draw rects as tracker moves
    if success:
        
        # Sucess on tracking
        pts1=(x,y)
        pts2=(x+w,y+h)
        cv2.rectangle(frame,pts1,pts2,(255,10,55),3)
        cv2.putText(frame,'Object',pts1,
                    cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
    # else
    else:
    
        # Failure on tracking
        cv2.putText(frame,'Fail to track the object',(150,200),
                    cv2.FONT_HERSHEY_SIMPLEX,1,(25,125,255),2)
        
    # Display Tracker
    cv2.putText(frame,
                "FIT.HUMG",
                (20,40),
                cv2.FONT_HERSHEY_SIMPLEX,1,
                (255,255,0),3)
    
    # Hiển thị kết quả theo vết và lưu ra file .mp4
    cv2.imshow("Demo Object Tracking",frame)    
    out.write(frame)
        
        
    # Exit with q button
    if cv2.waitKey(10) & 0xFF == ord("q"):
        break
    
    
# Giải phóng bộ nhớ:
cap.release()
out.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In [12]:
# Capture the Video
cap = cv2.VideoCapture('Video/crime-object.mp4')

# Tracker sử dụng OpenCV:
# tracker = cv2.TrackerCSRT_create() #CamShift
tracker=cv2.TrackerMIL_create()     #MeanShift

# Đọc frame đầu tiên:
ret,frame = cap.read()

#Thiết lập tham số ghi file Video:
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) + 0.5)
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) + 0.5)
size = (width, height)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('Result_object.mp4', fourcc, 20.0, size)


# Select our ROI
roi = cv2.selectROI(frame,False)

# Initialize tracker
ret = tracker.init(frame,roi)

while True:
    # Read the capture
    ret,frame = cap.read()
    
    # update tracker
    success, roi = tracker.update(frame)
    
    # roi -> from tuple to int
    (x,y,w,h)=tuple(map(int,roi))
    
    # Draw rects as tracker moves
    if success:
        
        # Sucess on tracking
        pts1=(x,y)
        pts2=(x+w,y+h)
        cv2.rectangle(frame,pts1,pts2,(255,10,55),3)
        cv2.putText(frame,'Object',pts1,
                    cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
    # else
    else:
    
        # Failure on tracking
        cv2.putText(frame,'Fail to track the object',(150,200),
                    cv2.FONT_HERSHEY_SIMPLEX,1,(25,125,255),2)
        
    # Display Tracker
    cv2.putText(frame,
                "FIT.HUMG",
                (20,40),
                cv2.FONT_HERSHEY_SIMPLEX,1,
                (255,255,0),3)
    
    # Hiển thị kết quả theo vết và lưu ra file .mp4
    cv2.imshow("Demo Object Tracking",frame)    
    out.write(frame)
        
        
    # Exit with q button
    if cv2.waitKey(10) & 0xFF == ord("q"):
        break
    
    
# Giải phóng bộ nhớ:
cap.release()
out.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1